In [ ]:
from GetPickles import *

In [ ]:
assert False

In [16]:
#normal 




import pickle 
import pandas as pd 
import os
import numpy as np 
import cv2
from keras.preprocessing import image 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import var
from keras.utils import to_categorical



def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
    
    # return the edged image
    return edged


def get_image_value(path, dim, edge = False, img_type = 'normal'): 
    if edge == False: 
        img = cv2.imread('../TestImages/AR.jpg')
        blurred = cv2.GaussianBlur(img, (3,3), 0)
        wide = cv2.Canny(blurred, 10,200)
        tight = cv2.Canny(blurred, 225, 250)
        auto = auto_canny(blurred)
        wide = cv2.resize(wide, dim, interpolation = cv2.INTER_CUBIC)
        tight = cv2.resize(tight, dim, interpolation = cv2.INTER_CUBIC)
        auto = cv2.resize(auto, dim, interpolation = cv2.INTER_CUBIC)
        return tight
    else: 
        img = image.load_img(path, target_size = dim)
        img = image.img_to_array(img)
        if img_type =='grey':
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = img.reshape(img.shape[0], img.shape[1], 1)
            return img
        else: 
            return img/255


def get_img_array(img_paths, dim, img_type, edge): 
    from tqdm import tqdm
    final_array = []
    for path in tqdm(img_paths): 
        img = get_image_value(path, dim, img_type, edge)
        final_array.append(img)
    final_array = np.array(final_array)
    if edge:
        return final_array.reshape(final_array.shape[0], dim[0], dim[1], 1)
    else: 
        return final_array

def get_pickles(nn_type, edge = False):
    
    if nn_type == 'normal': 
        DIM =  var.norm_dimension 
    elif nn_type == 'mobilenet': 
        DIM = var.mobilenet_dimension
    
    elif nn_type == 'inceptionnet': 
        DIM = var.inception_dimension
        
    elif nn_type == 'vgg16': 
        DIM = var.vgg_dimension
        
    
    pistol_paths = [f'../Separated/FinalImages/Pistol/{i}' for i in os.listdir('../Separated/FinalImages/Pistol')] 


    pistol_labels = [1 for i in range(len(pistol_paths))]

    rifle_paths = [f'../Separated/FinalImages/Rifle/{i}' for i in os.listdir('../Separated/FinalImages/Rifle')] 
    rifle_labels = [2 for i in range(len(rifle_paths))]

    negative = [f'../Separated/FinalImages/NoWeapon/{i}' for i in os.listdir('../Separated/FinalImages/NoWeapon')][:len(pistol_paths)]
    neg_labels = [0 for i in range(len(negative))]


    paths = pistol_paths + rifle_paths + negative
    labels = pistol_labels + rifle_labels + neg_labels


    x_train, x_test, y_train, y_test = train_test_split(paths, labels, stratify = labels, train_size = .90)

        
    new_x_train = get_img_array(x_train, DIM, img_type = var.img_type, edge = edge)
    new_x_test = get_img_array(x_test, DIM, img_type = var.img_type, edge = edge)
    
    print(pd.Series(y_train + y_test).value_counts())
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    
    if edge:
        pickle.dump(new_x_train, open(f'../Pickles/edge_{nn_type}_x_train.p', 'wb'), protocol=4)
        pickle.dump(y_train, open(f'../Pickles/edge_{nn_type}_y_train.p', 'wb'), protocol=4)
        pickle.dump(new_x_test, open(f'../Pickles/edge_{nn_type}_x_test.p', 'wb'), protocol=4)
        pickle.dump(y_test, open(f'../Pickles/edge_{nn_type}_y_test.p', 'wb'), protocol=4)
    else:
        pickle.dump(new_x_train, open(f'../Pickles/{nn_type}_x_train.p', 'wb'), protocol=4)
        pickle.dump(y_train, open(f'../Pickles/{nn_type}_y_train.p', 'wb'), protocol=4)
        pickle.dump(new_x_test, open(f'../Pickles/{nn_type}_x_test.p', 'wb'), protocol=4)
        pickle.dump(y_test, open(f'../Pickles/{nn_type}_y_test.p', 'wb'), protocol=4)
        
    
    
    
def get_samples(nn_type, edge = False): 
    if edge: 
        
        x_train = pickle.load(open(f'../Pickles/edge_{nn_type}_x_train.p', 'rb'))
        x_test = pickle.load(open(f'../Pickles/edge_{nn_type}_x_test.p', 'rb'))
        y_train = pickle.load(open(f'../Pickles/edge_{nn_type}_y_train.p', 'rb'))
        y_test = pickle.load(open(f'../Pickles/edge_{nn_type}_y_test.p', 'rb'))
    else: 
        x_train = pickle.load(open(f'../Pickles/{nn_type}_x_train.p', 'rb'))
        x_test = pickle.load(open(f'../Pickles/{nn_type}_x_test.p', 'rb'))
        y_train = pickle.load(open(f'../Pickles/{nn_type}_y_train.p', 'rb'))
        y_test = pickle.load(open(f'../Pickles/{nn_type}_y_test.p', 'rb'))
    
    y_test = to_categorical(y_test)
    y_train = to_categorical(y_train)

    return x_train, x_test, y_train, y_test





In [17]:




get_pickles('mobilenet')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 724/724 [00:08<00:00, 90.05it/s]


1    2933
0    2933
2    1370
dtype: int64


In [18]:
x_train, x_test, y_train, y_test = get_samples('mobilenet')
print(x_train.shape, x_test.shape) 
print(y_train.shape, y_test.shape)

(6512, 224, 224, 3) (724, 224, 224, 3)
(6512, 3) (724, 3)


In [ ]:
def get_image_value(path, dim, edge = False, img_type = 'normal'): 
    if edge: 
        img = cv2.imread('../TestImages/AR.jpg')
        blurred = cv2.GaussianBlur(img, (3,3), 0)
        wide = cv2.Canny(blurred, 10,200)
        tight = cv2.Canny(blurred, 225, 250)
        auto = auto_canny(blurred)
        wide = cv2.resize(wide, dim, interpolation = cv2.INTER_CUBIC)
        tight = cv2.resize(tight, dim, interpolation = cv2.INTER_CUBIC)
        auto = cv2.resize(auto, dim, interpolation = cv2.INTER_CUBIC)
        return tight
    else: 
        img = image.load_img(path, target_size = dim)
        img = image.img_to_array(img)
        
        if img_type =='grey':
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = img.reshape(img.shape[0], img.shape[1], 1)
            return img
        else: 
            return img/255

test = get_image_value('../TestImages/AR.jpg', var.mobilenet_dimension, img_type = 'mobilenet')
test.shape

In [ ]:
# get_pickles('inceptionnet')

In [ ]:
get_pickles('vgg16')